In [72]:
import cv2
import matplotlib.pyplot as plt
import pytesseract
import re
import pandas as pd
from PIL import Image
import io
import os

# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types

plt.figure(figsize=(40, 20))
plt.style.use('dark_background')

<Figure size 2880x1440 with 0 Axes>

In [247]:
server = '1716'
type = 'power' # or name

n = 4

In [142]:
#power
for i in range(50):
    n = i + 1
    initial = Image.open('data/' + server + '/original/' + str(n) + '.png')
    width = initial.size[0]
    height = initial.size[1]
    
    croped_power = initial.crop((width*0.72, height*0.27, width*0.85, height*0.94))
    croped_power.save('data/' + server + '/power/' + str(n) + '.png')

    img_ori = cv2.imread('data/' + server + '/power/' + str(n) + '.png')

    gray = cv2.cvtColor(img_ori, cv2.COLOR_BGR2GRAY)
    structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

    imgTopHat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, structuringElement)
    imgBlackHat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, structuringElement)

    imgGrayscalePlusTopHat = cv2.add(gray, imgTopHat)
    gray = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

    img_blurred = cv2.GaussianBlur(gray, ksize=(5, 5), sigmaX=0)

    ret, img_thresh = cv2.threshold(
        img_blurred, 
        180,
        255,
        cv2.THRESH_BINARY_INV
    )

    cv2.imwrite('data/' + server + '/power/' + str(n) + '.png', img_thresh)

In [158]:
li = list()
for i in range(50):
    n = i + 1
    detect_text('data/' + server + '/power/' + str(n) + '.png')

['120,832,558', '107,927,795', '76,411,001', '72,585,749', '70,447,776', '70,016,732', '69,102,127', '68,177,191', '66,759,729', '66,106,170', '65,716,326', '64,952,051', '62,171,213', '61,766,725', '61,424,774', '60,968,427', '60,926,984', '60,604,815', '60,425,209', '55,981,784', '54,545,473', '54,299,967', '54,045,813', '52,931,609', '49,364,560', '49,261,092', '48,363,317', '48,109,440', '47,740,325', '46,914,457', '46,892,966', '46,851,260', '46,736,145', '46,567,510', '44,713,256', '44,583,699', '43,638,061', '42,246,440', '42,225,917', '42,021,130', '41,566,415', '41,535,646', '41,385,769', '40,900,693', '40,415,483', '40,395,887', '40,385,002', '40,338,569', '40,137,162', '39,766,656', '39,242,834', '39,175,722', '38,520,982', '38,464,926', '38,224,315', '37,479,704', '37,260,768', '37,259,290', '36,855,699', '36,786,651', '36,411,221', '35,685,823', '35,627,791', '35,598,987', '35,248,506', '35,108,027', '35,028,235', '34,912,577', '34,903,691', '34,817,213', '34,604,306', '34

In [248]:
#name crop
for i in range(50):
    n = i + 1
    initial = Image.open('data/' + server + '/original/' + str(n) + '.png')
    width = initial.size[0]
    height = initial.size[1]
    
    croped_name = initial.crop((width*0.21, height*0.26, width*0.45, height*0.94))
    croped_name.save('./data/' + server + '/name/' + str(n) + '.png')

    img_ori = cv2.imread('data/' + server + '/name/' + str(n) + '.png')

    gray = cv2.cvtColor(img_ori, cv2.COLOR_BGR2GRAY)
    structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

    imgTopHat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, structuringElement)
    imgBlackHat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, structuringElement)

    imgGrayscalePlusTopHat = cv2.add(gray, imgTopHat)
    gray = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

    img_blurred = cv2.GaussianBlur(gray, ksize=(5, 5), sigmaX=0)

    ret, img_thresh = cv2.threshold(
        img_blurred, 
        180,
        255,
        cv2.THRESH_BINARY_INV
    )

    cv2.imwrite('data/' + server + '/name/' + str(n) + '.png', img_thresh)


In [251]:
# 이름 처리
p = re.compile('[\(\)\[\]]')

name = list()
allience = list()

for i in range(50):
    n = i+1
    li = list()
    detect_text('data/' + server + '/name/' + str(n) + '.png')

    for j in range(len(li)):

        if len(li) == 0:
            break

        target = li.pop(0)
        if p.match(target) != None:
            name.append(target)
            allience.append(li.pop(0))
        else:
            name.append(target)
            allience.append('없음')


In [252]:
df = pd.DataFrame({'name':name, 'Allience':allience})
df.to_excel('test.xlsx')

In [ ]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    # [START vision_python_migration_text_detection]
    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations

    s = ""
    
    for c in texts[0].description:
        if c != '\n':
            s += c
        else:
            li.append(s)
            s = ""

In [253]:
li = list()
detect_text('data/1716/name/10.png')
print(li)

['[16ZO] 희 니', 'Troop Zero', '[16ZO] -최강불때-', 'Troop Zero', '|', '[16ZO] 안녕따님', 'Troop Zero', '[16ZR] Valkilmor2', 'Red Moon', '[16ZO] 希凱是我兒', 'Troop Zero', '[16ZO] 옥골이', 'Troop Zero']
